In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split # used for splitting training and testing data

torch.manual_seed(1)

In [73]:
plt.figure(figsize=(8,5))

# time steps per batch of data
seq_length = 100

time_steps = np.linspace(0, np.pi, seq_length + 1)

data = pd.read_csv('../Datasets/SectionData.csv')

x = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

arr = data.iloc[:,:].values

<Figure size 576x360 with 0 Axes>

In [82]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    n_batches = len(arr)//batch_size_total
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    
    # Reshape into batch_size rows
    arr.reshape((batch_size, -1))
    
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

In [83]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, n_layers, hidden_size, drop_prob):
        
        super(NeuralNetwork, self).__init__()
        self.input_size = input_size
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.drop_prob = drop_prob
                
        self.lstm = nn.LSTM(input_size, hidden_size[0], n_layers, batch_first=True)
        
        self.dropout = nn.Dropout(drop_prob)
        
        # First fully connected layer
        self.fc1 = nn.Linear(hidden_size[0], hidden_size[1])
        # Second fully connected layer
        self.fc2 = nn.Linear(hidden_size[1], hidden_size[2])
        
        # Softmax activation
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x, hidden):
        # x(batch_size, seq_length, input_size)
        # hidden(n_layers, batch_size, hidden_dim)
        # r_out(batch_size, time_step, hidden_size)
        
        # Use x.view(seq_len, batch, input_size)
        # And this order corresponde to:
        # seq_len = time dimension (how many time steps do you have)
        # batch = mini-batch dimension (how much do you split ur data)
        # input_size = feature dimension (so how many features do you have)
        
        x = x.view(1, x.size(0), self.input_size)
        
        # get LSTM outputs
        lstm_out, hidden = self.lstm(x, hidden)

        # shape output to be (batch_size*seq_length, hidden_dim)
        lstm_out = lstm_out.view(-1, self.hidden_size[0])
        
        # dropout layer
        out = self.dropout(lstm_out)
        
        # get output of first fully connected layer
        out1 = self.fc1(out)
        
        # get output of second fully connected layer
        out2 = self.fc2(out1)
        
        # softmax activation function
        output = self.softmax(out2)
        
        # TODO: might need to reshape the out2
                
        return output, hidden
    
    def init_hidden(self, batch_size):
        '''initialize hidden states'''
        # initialized initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_size).zero_(),
                 weight.new(self.n_layers, batch_size, self.hidden_size).zero_())
        
        return hidden

In [84]:
def formatDataX(data):
    temp = torch.tensor(data).float()
    return temp.view(temp.size(0), 1, temp.size(1))

def formatDataY(data):
    return torch.tensor(data).float()

In [85]:
def one_hot_encode(arr, n_labels):

    
    # Initialize the the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

def preprocess(df):

    # drop ecgNum row
    df = df.drop(["ecgNum"], axis=1)
    # Classify the dependent and independent variables
    X = df.iloc[:, :-1].values
    Y = np.array([np.insert(np.zeros(5), label, 1) for label in df.iloc[:, -1].values])
    
    # split the data into train, validate, test
    X_temp, X_test, Y_temp, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
    X_train, X_val, Y_train, Y_val = train_test_split(X_temp, Y_temp, test_size=0.2, random_state=0)
    
    return X_train, X_val, X_test, Y_train, Y_val, Y_test

In [86]:
# size of the input at each time step
# TODO: handle input sizes
input_size = 100
# size of the hidden state and cell state at each time step
hidden_size = [64, 32, 6]
# number of LSTM layers
n_layers = 1
# dropout probability
drop_prob = 0.5

# instantiate the NN
neuralNet = NeuralNetwork(input_size, n_layers, hidden_size, drop_prob)
print(neuralNet)

NeuralNetwork(
  (lstm): LSTM(100, 64, batch_first=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=64, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=6, bias=True)
  (softmax): Softmax(dim=1)
)


In [87]:
# Mean Squared Error and Adam Optimizer with a learning rate of 0.01
# TODO: play with LR
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(neuralNet.parameters(), lr=0.01)

In [90]:
# train the NN
def train(neuralNet, n_steps, print_every, clip):
    # initialize the hidden state
    # hidden = torch.from_numpy(np.zeros((100,1,1,64))).float()
    
    X_train, X_val, X_test, Y_train, Y_val, Y_test = preprocess(data)
    
    batch_size = 64
    epochs = 100
    counter = 0
    for epoch in range(epochs):
        
        for x, y in get_batches(arr, batch_size, seq_length):
            counter += 1
            print(x)
        
            hidden = neuralNet.init_hidden(batch_size)

            # zero gradients
            net.zero_grad()

            # output from the neural net
            prediction, hidden = neuralNet(x, hidden)

            # print(prediction)
            # print(Y_train)

            #hidden = hidden.data

            hidden = tuple([each.data for each in hidden])

            # calculate the loss
            loss = criterion(prediction, y)

            # backpropogate and update weights
            loss.backward()

            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(neuralNet.parameters(), clip)
            optimizer.step()

            # display loss and predictions
            if epoch%print_every == 0:
                print('Loss ', loss.item())
                #plt.plot(time_steps[1:], x, 'r.') #input
                #plt.plot(time_steps[1:], predction.data.numpy().flatten(), 'b.') #predictions
            
    return neuralNet

In [91]:
n_steps = 100
print_every = 15
clip = 5

trained_NN = train(neuralNet, n_steps, print_every, clip)

[[-0.00761507 -0.01189971 -0.0125586  ... -0.01476791 -0.01129036
  -0.00685251]
 [-0.00803478 -0.01398042 -0.01742379 ... -0.00404229 -0.00141799
   0.00380178]
 [-0.0008748  -0.00907077 -0.00801822 ...  0.0008022   0.00386643
   0.01069081]
 ...
 [ 0.1710969   0.19036874  0.1733422  ...  0.09578539  0.10768675
   0.08889395]
 [ 0.11409358  0.08877038  0.09443073 ...  0.20060768  0.20112113
   0.20432174]
 [ 0.16453222  0.15079839  0.12953867 ...  0.12392653  0.13601824
   0.13548012]]


TypeError: new(): argument 'size' must be tuple of ints, but found element of type list at pos 3